In [1]:
#Import required libraries
import numpy as np
import pandas as pd

# Read the transaction_log file
transactions_1 = pd.read_csv('transactions.csv')
transactions_2 = pd.read_csv('transactions_v2.csv')
transactions = transactions_1.append(transactions_2).reset_index(drop=True)
transactions = transactions.loc[transactions['transaction_date'] >= 20170101]
transactions = transactions.sort_values(['msno','transaction_date','membership_expire_date'])

#transactions.info()

# Aggregate columns
Avg_payment_plan_days = transactions.groupby('msno', as_index=False)['payment_plan_days'].mean().reset_index(drop=True)
Avg_actual_amount_paid = transactions.groupby('msno', as_index=False)['actual_amount_paid'].mean().reset_index(drop=True)
Avg_plan_list_price = transactions.groupby('msno',as_index=False)['plan_list_price'].mean().reset_index(drop=True)
Number_of_transaction = transactions.groupby('msno',as_index=False)['payment_method_id'].count()
Number_of_transaction.rename(index=str,columns={'payment_method_id':'total_transactions'})

# Extracting the most recent transaction log for a user
transaction_intermediate = transactions.loc[transactions.groupby('msno').transaction_date.idxmax()].reset_index(drop=True)

# Combine columns
transaction_intermediate1 = pd.merge(transaction_intermediate,Avg_payment_plan_days,on='msno')
transaction_intermediate2 = pd.merge(transaction_intermediate1,Avg_plan_list_price,on='msno')
transaction_intermediate3 = pd.merge(transaction_intermediate2,Avg_actual_amount_paid,on='msno')
transaction_final = pd.merge(transaction_intermediate3,Number_of_transaction,on='msno')

# Rename columns
transaction_final = transaction_final.rename(index=str,columns={'payment_plan_days_y':'Avg_payment_plan_days','plan_list_price_y':'Avg_plan_list_price','actual_amount_paid_y':'Avg_actual_amount_paid','payment_plan_days_x':'latest_payment_plan_days','plan_list_price_x':'latest_plan_list_price','actual_amount_paid_x':'latest_actual_amount_paid','transaction_date':'latest_transaction_date','membership_expire_date':'latest_membership_expire_date','is_cancel':'latest_is_cancel','is_auto_renew':'latest_is_auto_renew'})

transaction_final = transaction_final.reset_index(drop=True)

# Outputting results to a csv file
transaction_final.to_csv('transactions_log_initial.csv')